# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [38]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'NCAA_matchup_classifier'

experiment=Experiment(ws, experiment_name)

# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster,", amlcompute_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                                           #vm_priority = 'lowpriority',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [27]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
#early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling(
    {
        "C": uniform(1.5, 5.5),
        "max_iter": choice(500, 1000, 2000),
        "penalty": choice(["l1", "l2"]),
        "solver": choice(["liblinear", "saga"])
    }
)

#TODO: Create your estimator and hyperdrive config
env = Environment.get(ws, name="AzureML-AutoML")
estimator = ScriptRunConfig(source_directory=".",
                            script="train.py",
                            compute_target=compute_target,
                            environment=env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                         hyperparameter_sampling=param_sampling,
                                         primary_metric_name="Accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=50,
                                         max_concurrent_runs=5)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:80.


In [4]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [43]:
best_hyper_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hyper_run_metrics = best_hyper_run.get_metrics()

print("ID: ", best_hyper_run.id)
best_hyper_run_metrics

ID:  HD_46499023-5d80-419a-86e3-e703790470c2_13


{'Regularization:': 5.460777567453194,
 'Max Iterations:': '1000',
 'Penalty:': 'l1',
 'Solver:': 'liblinear',
 'Scores': 'array([0.85714286, 0.84920635, 0.87301587, 0.85714286, 0.84126984,\n       0.8968254 , 0.88095238, 0.85714286, 0.92063492, 0.84126984])',
 'Accuracy': 0.8674603174603176}

In [44]:
#TODO: Save the best model
model = best_hyper_run.register_model(model_name="hyperdrive_best_model",
                                      model_path="./outputs/hd_model.pkl",
                                      properties=best_hyper_run_metrics,
                                      model_framework=Model.Framework.SCIKITLEARN)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
inf_config = InferenceConfig(environment=env, entry_script="entry.py")

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=2,
    auth_enabled=True,
    enable_app_insights=True
)

service = Model.deploy(
    workspace=ws,
    name="hyper1",
    models=[model],
    inference_config=inf_config,
    deployment_config=aci_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-11 23:33:33+00:00 Creating Container Registry if not exists.
2021-08-11 23:33:34+00:00 Registering the environment.
2021-08-11 23:33:34+00:00 Use the existing image.
2021-08-11 23:33:35+00:00 Generating deployment configuration.
2021-08-11 23:33:36+00:00 Submitting deployment to compute.
2021-08-11 23:33:39+00:00 Checking the status of deployment hyper1.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

uri = service.scoring_uri
key, _ = service.get_keys()

headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

data = {
    "G1": 30,
    "G2": 32,
    "S1": 1,
    "S2": 1,
    "PF1": 17.1,
    "PF2": 16.3,
    "3P%1": 0.384,
    "3P%2": 0.332,
    "TOV1": 11.5,
    "TOV2": 12.0,
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [46]:
print(service.get_logs())

#service.delete()

2021-08-11T23:22:29,599035700+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-08-11T23:22:29,597370800+00:00 - iot-server/run 
2021-08-11T23:22:29,623990300+00:00 - rsyslog/run 
2021-08-11T23:22:29,670230200+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_3489174eb648a475780c9959ff366072/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-11T23:22:30,078068200+00:00 - iot-server/finish 1 0
2021-08-11T23:22:30,084497300+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (212)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 241
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-11 23:22:31,713 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found

In [ ]:
Model.get_model_path("hyperdrive_best_model", 1, ws)